In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/train/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/test/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [4]:
resnet101 = tf.keras.applications.resnet.ResNet101(include_top=False, weights=None,input_shape=(128, 128, 3), pooling='avg')
x1 = tf.keras.layers.Dense(1, activation='sigmoid')(resnet101.output)
resnet101_model = tf.keras.Model(resnet101.input, x1)

In [5]:
METRICS = [
        'accuracy',
    ]
    
resnet101_model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=METRICS
    )

In [6]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [7]:
history = resnet101_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[lr_scheduler]
)

Epoch 1/10
652/652 [==============================] - 212s 217ms/step - loss: 0.6927 - accuracy: 0.7316 - val_loss: 0.6203 - val_accuracy: 0.6042 - lr: 0.0100
Epoch 2/10
652/652 [==============================] - 110s 168ms/step - loss: 0.3609 - accuracy: 0.8290 - val_loss: 0.9396 - val_accuracy: 0.3814 - lr: 0.0089
Epoch 3/10
652/652 [==============================] - 111s 170ms/step - loss: 0.3072 - accuracy: 0.8685 - val_loss: 1.2859 - val_accuracy: 0.3750 - lr: 0.0079
Epoch 4/10
652/652 [==============================] - 109s 167ms/step - loss: 0.2790 - accuracy: 0.8794 - val_loss: 0.8545 - val_accuracy: 0.7163 - lr: 0.0071
Epoch 5/10
652/652 [==============================] - 111s 170ms/step - loss: 0.2419 - accuracy: 0.9020 - val_loss: 0.6249 - val_accuracy: 0.7147 - lr: 0.0063
Epoch 6/10
652/652 [==============================] - 110s 168ms/step - loss: 0.2225 - accuracy: 0.9155 - val_loss: 0.5084 - val_accuracy: 0.7965 - lr: 0.0056
Epoch 7/10
652/652 [==========================

In [8]:
resnet101_model.save("ResNet101_xray_Pneumonia.h5")